# Hooks
Hooks are simple functions that can be registered to be called during the forward or backward pass of a `nn.Module`.
These functions can be used to print out information or modify the module.

Here is a simple forward hook example that prints some information about the input and output of a module.

Tip: Don't forget to remove the hook afterwards!


In [ ]:
def tensorinfo_hook(module, input_, output):
    """
    Register this forward hook to print some infos about the tensor/module.
    """
    print(f"Inside '{module.__class__.__name__}' forward")
    print(f"  input:     {str(type(input_)):<25}")
    print(f"  input[0]:  {str(type(input_[0])):<25} {input_[0].size()}")
    print(f"  output:    {str(type(output)):<25} {output.data.size()}")
    print()

In [ ]:
import torch as pt
import torch.nn as nn

In [ ]:
model = nn.Linear(1, 3)

In [ ]:
hook = model.register_forward_hook(tensorinfo_hook)

In [ ]:
model(pt.rand(1));

In [ ]:
hook.remove()

In [ ]:
model(pt.rand(1));

## Exercise
- Write a context manager hook that removes the hook when leaving the with block.

# `torch.jit`

Eager execution is great for development and debugging. but it can be hard to (automatically) optimize the code and deploy it.

Now there is`torch.jit` with two flavours:

- `torch.jit.trace` does not record control flow.
- `torch.jit.script` records control flow and creates an intermediate representation that can be optimized; only supports a subset of Python.

Note: don't forget `model.eval()` and `model.train()`.

## Init, helpers, utils, ...

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [ ]:
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
from IPython.core.debugger import set_trace

# `torch.jit.trace`

In [ ]:
def f(x):
    if x.item() < 0:
        return torch.tensor(0)
    else:
        return x

In [ ]:
f(torch.tensor(-1))

In [ ]:
f(torch.tensor(3))

In [ ]:
X = torch.tensor(1)
traced = torch.jit.trace(f, X)

In [ ]:
type(traced)

In [ ]:
traced(torch.tensor(1))

In [ ]:
traced.graph

In [ ]:
traced(torch.tensor(-1))

## Storing and restoring

In [ ]:
traced.save("traced.pt")

In [ ]:
!ls -ltr *.pt

In [ ]:
g = torch.jit.load("traced.pt")

In [ ]:
g(torch.tensor(1))

In [ ]:
g(torch.tensor(-1))

# `torch.jit.script`

In [ ]:
bool(torch.tensor(1) < 2)

In [ ]:
@torch.jit.script
def f(x):
    if bool(x < 0):
        result = torch.zeros(1)
    else:
        result = x
    return result

This is `torchscript` which is a only a subset of python.

In [ ]:
f(torch.tensor(-1))

In [ ]:
f(torch.tensor(1))

In [ ]:
type(f)

In [ ]:
f.graph

## Storing and restoring

In [ ]:
torch.jit.save(f, "scripted.pt")

In [ ]:
!ls -ltr scripted.pt

In [ ]:
g = torch.jit.load("scripted.pt")

In [ ]:
g(torch.tensor(-1))

In [ ]:
g(torch.tensor(1))

## Subclassing `torch.jit.ScriptModule`
If you work with `nn.Module` replace it by `torch.jit.ScriptModule` (see [[tutorial]](https://pytorch.org/tutorials/beginner/deploy_seq2seq_hybrid_frontend_tutorial.html) for more).

```python
class MyModule(torch.jit.ScriptModule):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        # ...
        return x
```

# PyTorch and C++

PyTorch offers a very nice(!) C++ interface which is very close to Python.

## Loading traced models from C++

```c++
#include <torch/script.h>

int main(int(argc, const char* argv[]) {
    auto module = torch::jit::load("scrpted.pt");
    // data ...
    module->forward(data);
}
```

Copyright 2021 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.